In [2]:
#@title Research Analyzer
#@markdown This is part of the reasearch analyzer project

In [ ]:
#@title Installing dependancies
%%capture
!pip install marker-pdf
!pip install gradio
!pip install scidownl
!pip install langchain
!pip install langchain-google-genai
!pip install langchain-groq
!git clone https://github.com/mohamedfadlalla/Research-Analyzer.git


In [5]:
#@title Setup
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate


import re
import os 

from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq



# setup google gemini api keys
import os
from dotenv import load_dotenv
import platform

if platform.system() == "Windows":
    load_dotenv()
    GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
    GROQ_API_KEY = os.getenv("GROQ_API_KEY")
elif platform.system() == "Linux":
    import sys
    sys.path.append('/content/Research-Analyzer')
    os.chdir('/content/Research-Analyzer')
    from google.colab import userdata
    GOOGLE_API_KEY = userdata.get('GOOGLE_AI_STUDIO2')
    GROQ_API_KEY = userdata.get('GROQ_API_KEY')
else:
    raise OSError("Unsupported operating system")


pro = ChatGoogleGenerativeAI(model="gemini-1.5-pro", google_api_key=GOOGLE_API_KEY, temperature=0.4, convert_system_message_to_human=True)
flash = ChatGoogleGenerativeAI(model="gemini-1.5-flash", google_api_key=GOOGLE_API_KEY, temperature=0.3, convert_system_message_to_human=True)

llm = ChatGroq(
    temperature=0,
    # model_name="mixtral-8x7b-32768",
    # model_name="llama3-70b-8192",
    model_name="llama-3.1-70b-versatile",
    # model_name="mixtral-8x7b-32768",
    # model_name="llama3-8b-8192",
    # model_name="llama-3.1-8b-instant",
    # model_name="gemma2-9b-it",
    
    groq_api_key=GROQ_API_KEY 
)

import gradio as gr
import summary  # Make sure the 'summary' module is in your Python path
import functions  # Make sure the 'functions' module is in your Python path


# Define the function to summarize the file, generate the workflow graph, and save the results to 'summary.md'
def process_file(file):
    if file is None:
        return "No file uploaded", None, None
    
    file_path = file.name
    results = summary.summarize(file_path, llm)
    
    # Save the results to summary.md
    with open("summary.md", "w") as f:
        f.write(results)
    
    # Generate the workflow graph
    functions.get_workflow(file_path, llm)
    
    return results, "summary.md", "graph.png"




In [ ]:
#@title Launch the App




interface = gr.Interface(fn=process_file, inputs=gr.File(label="Upload File"), outputs=[gr.Markdown(),
                                                                      gr.File(),
                                                                      gr.Image()
                                                                     ])
interface.launch()
